# Entropic OTC

This notebook aims to compare the results of entropic OTC and exact OTC, demonstrating that the entropic OTC converges to the exact OTC. Two examples are provided to illustrate this convergence.

In [1]:
import numpy as np
import networkx as nx
import sys
import os
import json

sys.path.append(os.path.abspath("../src"))

from pyotc.otc_backend.policy_iteration.dense.exact import exact_otc
from pyotc.otc_backend.policy_iteration.sparse.exact import (
    exact_otc as exact_otc_sparse,
)
from pyotc.otc_backend.policy_iteration.dense.entropic import entropic_otc
from pyotc.otc_backend.graph.utils import adj_to_trans, get_degree_cost, get_sq_cost

from pyotc.examples.wheel import wheel_1, wheel_2, wheel_3
from pyotc.examples.stochastic_block_model import stochastic_block_model

## Example 1: Wheel Graphs

In [2]:
# Create adjacency matrices of wheel graphs
A1 = nx.to_numpy_array(wheel_1)
A2 = nx.to_numpy_array(wheel_2)
A3 = nx.to_numpy_array(wheel_3)

# Convert adjacency matrices to transition matrices
P1 = adj_to_trans(A1)
P2 = adj_to_trans(A2)
P3 = adj_to_trans(A3)

# Obtain degree based costs for the wheel graphs
c12 = get_degree_cost(A1, A2)
c13 = get_degree_cost(A1, A3)

### Compute the exact OTC costs between the wheel graphs

In [3]:
wheel_exact12, _, _ = exact_otc(P1, P2, c12)
wheel_exact13, _, _ = exact_otc(P1, P3, c13)

print("\nExact OTC cost between Wheel 1 and Wheel 2:", wheel_exact12)
print("Exact OTC cost between Wheel 1 and Wheel 3:", wheel_exact13)

Starting exact_otc_dense...
Iteration: 0
Computing exact TCE...
Computing exact TCI...
Iteration: 1
Computing exact TCE...
Computing exact TCI...
Convergence reached in 2 iterations. Computing stationary distribution...
[exact_otc] Finished. Total time elapsed: 0.086 seconds.
Starting exact_otc_dense...
Iteration: 0
Computing exact TCE...
Computing exact TCI...
Iteration: 1
Computing exact TCE...
Computing exact TCI...
Convergence reached in 2 iterations. Computing stationary distribution...
[exact_otc] Finished. Total time elapsed: 0.054 seconds.

Exact OTC cost between Wheel 1 and Wheel 2: 2.655172413793098
Exact OTC cost between Wheel 1 and Wheel 3: 2.5517241379310294


### Compute the entropic OTC costs between the wheel graphs

- L = 25, T = 50, xi = 0.1, sink_iter = 10

In [4]:
wheel_entropic12, _, _ = entropic_otc(
    P1, P2, c12, get_sd=True, L=25, T=50, xi=0.1, sink_iter=100, method="logsinkhorn"
)
wheel_entropic13, _, _ = entropic_otc(
    P1, P3, c13, get_sd=True, L=25, T=50, xi=0.1, sink_iter=100, method="logsinkhorn"
)

print("Entropic OTC cost between Wheel 1 and Wheel 2:", wheel_entropic12)
print("Entropic OTC cost between Wheel 1 and Wheel 3:", wheel_entropic13)

Entropic OTC cost between Wheel 1 and Wheel 2: 2.6552724778475816
Entropic OTC cost between Wheel 1 and Wheel 3: 2.55387422375081


## Example 2: Stochastic Block Models

In [5]:
# Seed number
np.random.seed(100)

# Generate two stochastic block model graphs with 4 blocks, each containing 5 nodes
m = 5
A1 = stochastic_block_model(
    (m, m, m, m),
    np.array(
        [
            [0.9, 0.1, 0.1, 0.1],
            [0.1, 0.9, 0.1, 0.1],
            [0.1, 0.1, 0.9, 0.1],
            [0.1, 0.1, 0.1, 0.9],
        ]
    ),
)

A2 = stochastic_block_model(
    (m, m, m, m),
    np.array(
        [
            [0.9, 0.1, 0.1, 0.1],
            [0.1, 0.9, 0.1, 0.1],
            [0.1, 0.1, 0.9, 0.1],
            [0.1, 0.1, 0.1, 0.9],
        ]
    ),
)

# Convert adjacency matrices to transition matrices
P1 = adj_to_trans(A1)
P2 = adj_to_trans(A2)

# Obtain degree based costs
c = get_degree_cost(A1, A2)

### Compute the exact OTC costs between SBMs

In [6]:
sbm_exact, _, _ = exact_otc(P1, P2, c)
print("\nExact OTC cost between SBM1 and SBM2:", sbm_exact)

Starting exact_otc_dense...
Iteration: 0
Computing exact TCE...
Computing exact TCI...
Iteration: 1
Computing exact TCE...
Computing exact TCI...
Iteration: 2
Computing exact TCE...
Computing exact TCI...
Iteration: 3
Computing exact TCE...
Computing exact TCI...
Iteration: 4
Computing exact TCE...
Computing exact TCI...
Convergence reached in 5 iterations. Computing stationary distribution...
[exact_otc] Finished. Total time elapsed: 0.267 seconds.

Exact OTC cost between SBM1 and SBM2: 1.1247533069245113


### Compute the entropic OTC costs between SBMs

- L = 25, T = 50, xi = 10, sink_iter = 1000

In [7]:
sbm_entropic, _, _ = entropic_otc(
    P1, P2, c, get_sd=True, L=25, T=50, xi=10, sink_iter=1000, method="logsinkhorn"
)

print("Entropic OTC cost between SBM1 and SBM2:", sbm_entropic)

Entropic OTC cost between SBM1 and SBM2: 1.1398150358827743


## Example 3: Molecular Graphs

To demonstrate the efficiency of entropic method on larger networks, we use the examples of two organic molecules. Moreover, previous examples compared the graphs of the same size and cost was dependent upon the adjacency structure. This molecular graph example is comparing two graphs of different sizes, and cost is defined by the node-level features, which is similar to real world network data.
Below is the visulization:

![Molecular Graph](../tests/benchmark/graph%20data/molecule_visualization.png)

**Notes:**
- This figure illustrates how molecules are represented as **weighted graphs**.  
- **Nodes** represent atoms, and **edges** represent chemical bonds. With this representation, a weighted adjacency matrix can be constructed.
- In convention:  
  - Single bond → edge weight **1**  
  - Double bond → edge weight **2**  
  - Triple bond → edge weight **3**  
  - Bonds within a benzene ring → edge weight **1.5**  
- We use the **atomic number** as the continuous node label, and hence adopt the cost function \[c(u,v) = (u - v)^2\].

In [8]:
# load stored info of molecule1
with open(
    os.path.abspath("../tests/benchmark/graph data/graph_G1.json"),
    "r",
    encoding="utf-8",
) as f:
    data = json.load(f)
A1 = np.array(data["A"])  # weighted adjacency matrix of molecule 1
# P1 = np.array(data["P"])
V1 = np.array(data["V"])  # node label vector of molecule 1

# load stored info of molecule2
with open(
    os.path.abspath("../tests/benchmark/graph data/graph_G2.json"),
    "r",
    encoding="utf-8",
) as f:
    data = json.load(f)
A2 = np.array(data["A"])  # weighted adjacency matrix of molecule 2
# P2 = np.array(data["P"])
V2 = np.array(data["V"])  # node label vector of molecule 2

P1, P2 = adj_to_trans(A1), adj_to_trans(A2)  # obtain corresponding transition matrices
c = get_sq_cost(V1, V2)

In [9]:
d0, _, _ = exact_otc(P1, P2, c)

Starting exact_otc_dense...
Iteration: 0
Computing exact TCE...
Computing exact TCI...
Iteration: 1
Computing exact TCE...
Computing exact TCI...
Iteration: 2
Computing exact TCE...
Computing exact TCI...
Iteration: 3
Computing exact TCE...
Computing exact TCI...
Iteration: 4
Computing exact TCE...
Computing exact TCI...
Iteration: 5
Computing exact TCE...
Computing exact TCI...
Convergence reached in 6 iterations. Computing stationary distribution...
[exact_otc] Finished. Total time elapsed: 43.867 seconds.


In [10]:
d00, _, _ = exact_otc_sparse(P1, P2, c)

Starting exact_otc_sparse...
Iteration: 0
Computing exact TCE...
Computing exact TCI...
Iteration: 1
Computing exact TCE...
Computing exact TCI...
Iteration: 2
Computing exact TCE...
Computing exact TCI...
Iteration: 3
Computing exact TCE...
Computing exact TCI...
Iteration: 4
Computing exact TCE...
Computing exact TCI...
Iteration: 5
Computing exact TCE...
Computing exact TCI...
Iteration: 6
Computing exact TCE...
Computing exact TCI...
Iteration: 7
Computing exact TCE...
Computing exact TCI...
Iteration: 8
Computing exact TCE...
Computing exact TCI...
Iteration: 9
Computing exact TCE...
Computing exact TCI...
Iteration: 10
Computing exact TCE...
Computing exact TCI...
Iteration: 11
Computing exact TCE...
Computing exact TCI...
Iteration: 12
Computing exact TCE...
Computing exact TCI...
Iteration: 13
Computing exact TCE...
Computing exact TCI...
Iteration: 14
Computing exact TCE...
Computing exact TCI...
Iteration: 15
Computing exact TCE...
Computing exact TCI...
Iteration: 16
Computi

In [11]:
d1, _, _ = entropic_otc(
    P1,
    P2,
    c,
    L=25,
    T=50,
    xi=0.1,
    sink_iter=1000,
    method="logsinkhorn",
    get_sd=False,
    silent=False,
)

Starting entropic otc with logsinkhorn method...
Iteration: 1
Computing entropic TCE...
Computing entropic TCE...
[Iter 1 taking 22.09s] Δg=8.281e-01, g[0]=0.171826, Δg/g[0]=4.819e+00, total elapsed=22.11s
Iteration: 2
Computing entropic TCE...
Computing entropic TCE...
[Iter 2 taking 21.60s] Δg=1.452e-03, g[0]=0.170374, Δg/g[0]=8.522e-03, total elapsed=43.71s
Iteration: 3
Computing entropic TCE...
Computing entropic TCE...
[Iter 3 taking 21.83s] Δg=1.407e-05, g[0]=0.170360, Δg/g[0]=8.258e-05, total elapsed=65.54s
Iteration: 4
Computing entropic TCE...
Computing entropic TCE...
[Iter 4 taking 22.31s] Δg=1.374e-07, g[0]=0.170360, Δg/g[0]=8.066e-07, total elapsed=87.85s
Convergence reached in 4 iterations. No stationary distribution computation requested.
[entropic_otc] Finished. Total time elapsed: 87.887 seconds.


In [12]:
d2, _, _ = entropic_otc(
    P1,
    P2,
    c,
    L=25,
    T=50,
    xi=0.1,
    sink_iter=1000,
    method="ot_sinkhorn",
    reg_num=0.1,
    get_sd=False,
    silent=False,
)

Starting entropic otc with ot_sinkhorn method...
Iteration: 1
Computing entropic TCE...
Computing entropic TCE...
[Iter 1 taking 1.09s] Δg=8.281e-01, g[0]=0.171826, Δg/g[0]=4.819e+00, total elapsed=1.11s
Iteration: 2
Computing entropic TCE...
Computing entropic TCE...
[Iter 2 taking 0.95s] Δg=-1.087e-02, g[0]=0.182692, Δg/g[0]=-5.947e-02, total elapsed=2.07s
Convergence reached in 2 iterations. No stationary distribution computation requested.
[entropic_otc] Finished. Total time elapsed: 2.101 seconds.


In [13]:
d3, _, _ = entropic_otc(
    P1,
    P2,
    c,
    L=25,
    T=50,
    xi=0.1,
    sink_iter=1000,
    method="ot_logsinkhorn",
    reg_num=0.1,
    get_sd=False,
    silent=False,
)

Starting entropic otc with ot_logsinkhorn method...
Iteration: 1
Computing entropic TCE...
Computing entropic TCE...
[Iter 1 taking 1.31s] Δg=8.281e-01, g[0]=0.171826, Δg/g[0]=4.819e+00, total elapsed=1.34s
Iteration: 2
Computing entropic TCE...
Computing entropic TCE...
[Iter 2 taking 1.26s] Δg=-1.087e-02, g[0]=0.182692, Δg/g[0]=-5.947e-02, total elapsed=2.60s
Convergence reached in 2 iterations. No stationary distribution computation requested.
[entropic_otc] Finished. Total time elapsed: 2.640 seconds.


In [14]:
d4, _, _ = entropic_otc(
    P1,
    P2,
    c,
    L=25,
    T=50,
    xi=0.1,
    sink_iter=1000,
    method="ot_greenkhorn",
    reg_num=0.1,
    get_sd=False,
    silent=False,
)

Starting entropic otc with ot_greenkhorn method...
Iteration: 1
Computing entropic TCE...
Computing entropic TCE...
[Iter 1 taking 1.04s] Δg=8.281e-01, g[0]=0.171826, Δg/g[0]=4.819e+00, total elapsed=1.06s
Iteration: 2
Computing entropic TCE...
Computing entropic TCE...
[Iter 2 taking 1.03s] Δg=-1.087e-02, g[0]=0.182692, Δg/g[0]=-5.947e-02, total elapsed=2.08s
Convergence reached in 2 iterations. No stationary distribution computation requested.
[entropic_otc] Finished. Total time elapsed: 2.119 seconds.


In [15]:
print(
    f"dense exact_otc calculated value: {d0},\n\
        sparse exact_otc calculated value: {d00},\n\
        entropic otc with 'logsinkhorn' calculated value: {d1},\n\
        entropic otc with 'ot_sinkhorn' calculated value:{d2},\n\
        entropic otc with 'ot_logsinkhorn' calculated value{d3},\n\
        entropic otc with 'ot_greenkhorn' calculated value{d4}"
)

dense exact_otc calculated value: 0.04964491760343259,
        sparse exact_otc calculated value: None,
        entropic otc with 'logsinkhorn' calculated value: 0.1703601635626674,
        entropic otc with 'ot_sinkhorn' calculated value:0.18269158660106682,
        entropic otc with 'ot_logsinkhorn' calculated value0.18269158660106677,
        entropic otc with 'ot_greenkhorn' calculated value0.18269158651810388
